In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from tensorflow import keras
from string import punctuation

In [3]:
lines = pd.read_csv('dataset/newyork/ArticlesMarch2018.csv')

In [4]:
lines['headline'].isnull().values.any()

False

In [5]:
cnt = 0
for line in lines['headline']:
    if line == 'Unknown':
        cnt += 1
print(cnt)

135


In [6]:
headline = [title for title in lines.headline.values if title != 'Unknown']

In [7]:
cnt = 0
for line in headline:
    if line == 'Unknown':
        cnt += 1
print(cnt)

0


In [8]:
def reprprocessing(s):
    s = s.encode('utf8').decode('ascii', 'ignore')
    return ''.join(c for c in s if c not in punctuation)

In [9]:
text = [reprprocessing(x) for x in headline]

In [10]:
text[1:5]

['US Advances Military Plans for North Korea',
 'Mr Trump and the Very Bad Judge',
 'To Erase Dissent China Bans Pooh Bear and N',
 'Loans Flowed to Kushner Cos After Visits to the White House']

In [11]:
t = keras.preprocessing.text.Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
vocab_size

3582

In [12]:
seqences = []
for line in text:
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        seqence = encoded[:i+1]
        seqences.append(seqence)
seqences[:11]

[[1119, 1120],
 [1119, 1120, 116],
 [1119, 1120, 116, 1121],
 [31, 1122],
 [31, 1122, 589],
 [31, 1122, 589, 392],
 [31, 1122, 589, 392, 7],
 [31, 1122, 589, 392, 7, 61],
 [31, 1122, 589, 392, 7, 61, 70],
 [117, 10],
 [117, 10, 6]]

In [13]:
index_to_word = {}
for key, value in t.word_index.items():
    index_to_word[value] = key
index_to_word[1], index_to_word[582]

('the', 'spending')

In [14]:
max_len = max(len(s) for s in seqences)
max_len

18

In [15]:
seqences = keras.preprocessing.sequence.pad_sequences(seqences, max_len)

In [16]:
X = seqences[:, :-1]
Y = seqences[:, -1]
y = keras.utils.to_categorical(Y, vocab_size)

In [17]:
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 10, input_length=max_len-1, name='Embedding_Layer'),
    keras.layers.GRU(128, name='GRU_Layer'),
    keras.layers.Dense(vocab_size, activation='softmax', name='Output')
])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 17, 10)            35820     
_________________________________________________________________
GRU_Layer (GRU)              (None, 128)               53760     
_________________________________________________________________
Output (Dense)               (None, 3582)              462078    
Total params: 551,658
Trainable params: 551,658
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile('adam', 'categorical_crossentropy', ['accuracy'])

In [20]:
model.fit(X, y, None, 200)

Train on 8057 samples
Epoch 1/200
8057/8057 [==============================] - 4s 463us/sample - loss: 7.7085 - accuracy: 0.0285
Epoch 2/200
8057/8057 [==============================] - 2s 280us/sample - loss: 7.1146 - accuracy: 0.0334
Epoch 3/200
8057/8057 [==============================] - 2s 280us/sample - loss: 6.9179 - accuracy: 0.0447
Epoch 4/200
8057/8057 [==============================] - 2s 280us/sample - loss: 6.7090 - accuracy: 0.0494
Epoch 5/200
8057/8057 [==============================] - 2s 281us/sample - loss: 6.4646 - accuracy: 0.0562
Epoch 6/200
8057/8057 [==============================] - 2s 280us/sample - loss: 6.1937 - accuracy: 0.0585
Epoch 7/200
8057/8057 [==============================] - 2s 280us/sample - loss: 5.9150 - accuracy: 0.0621
Epoch 8/200
8057/8057 [==============================] - 2s 280us/sample - loss: 5.6328 - accuracy: 0.0688
Epoch 9/200
8057/8057 [==============================] - 2s 282us/sample - loss: 5.3554 - accuracy: 0.0735
Epoch 10/200
80